# ADS 509 Module 1: APIs and Web Scraping

This notebook has two parts. In the first part, you will scrape lyrics from AZLyrics.com. In the second part, you'll run code that verifies the completeness of your data pull. 

For this assignment you have chosen two musical artists who have at least 20 songs with lyrics on AZLyrics.com. We start with pulling some information and analyzing them.


## General Assignment Instructions

These instructions are included in every assignment, to remind you of the coding standards for the class. Feel free to delete this cell after reading it. 

One sign of mature code is conforming to a style guide. We recommend the [Google Python Style Guide](https://google.github.io/styleguide/pyguide.html). If you use a different style guide, please include a cell with a link. 

Your code should be relatively easy-to-read, sensibly commented, and clean. Writing code is a messy process, so please be sure to edit your final submission. Remove any cells that are not needed or parts of cells that contain unnecessary code. Remove inessential `import` statements and make sure that all such statements are moved into the designated cell. 

Make use of non-code cells for written commentary. These cells should be grammatical and clearly written. In some of these cells you will have questions to answer. The questions will be marked by a "Q:" and will have a corresponding "A:" spot for you. *Make sure to answer every question marked with a `Q:` for full credit.* 


# Importing Libraries

In [3]:
import os
import datetime
import re

# for the lyrics scrape section
import requests
import time
from bs4 import BeautifulSoup
from collections import defaultdict, Counter
import random

In [12]:
# Use this cell for any import statements you add
import shutil


---

# Lyrics Scrape

This section asks you to pull data by scraping www.AZLyrics.com. In the notebooks where you do that work you are asked to store the data in specific ways. 

In [20]:
artists = {'jellyroll':"https://www.azlyrics.com/j/jellyroll.html",
           'linkinpark':"https://www.azlyrics.com/l/linkinpark.html"} 
# we'll use this dictionary to hold both the artist name and the link on AZlyrics

## A Note on Rate Limiting

The lyrics site, www.azlyrics.com, does not have an explicit maximum on number of requests in any one time, but in our testing it appears that too many requests in too short a time will cause the site to stop returning lyrics pages. (Entertainingly, the page that gets returned seems to only have the song title to [a Tom Jones song](https://www.azlyrics.com/lyrics/tomjones/itsnotunusual.html).) 

Whenever you call `requests.get` to retrieve a page, put a `time.sleep(5 + 10*random.random())` on the next line. This will help you not to get blocked. If you _do_ get blocked, which you can identify if the returned pages are not correct, just request a lyrics page through your browser. You'll be asked to perform a CAPTCHA and then your requests should start working again. 

## Part 1: Finding Links to Songs Lyrics

That general artist page has a list of all songs for that artist with links to the individual song pages. 

Q: Take a look at the `robots.txt` page on www.azlyrics.com. (You can read more about these pages [here](https://developers.google.com/search/docs/advanced/robots/intro).) Is the scraping we are about to do allowed or disallowed by this page? How do you know? 

A: It appears that webcrawlers other than "008" are allowed to access everything on the website outside of any URLs that begin with '/song/' or '/lyricsdb/'. so, I believe the actions we are about to perform are allowed.


In [53]:
# Let's set up a dictionary of lists to hold our links
lyrics_pages = defaultdict(list)

for artist, artist_page in artists.items():
    r = requests.get(artist_page)
    time.sleep(5 + 10 * random.random())

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(r.content, 'html.parser')

    # Find all links to song lyrics pages
    for link in soup.find_all('a', href=True):
        href = link['href']
        if 'lyrics/' in href and href.endswith('.html'):  # Only pick song lyrics pages
            # Check for relative URLs and construct the full URL accordingly
            if href.startswith('/'):
                full_url = "https://www.azlyrics.com" + href
            elif href.startswith('//'):
                full_url = "https:" + href
            elif href.startswith('..'):
                full_url = "https://www.azlyrics.com" + href[2:]
            else:
                full_url = href  # Already a full URL
            
            lyrics_pages[artist].append(full_url)

Let's make sure we have enough lyrics pages to scrape. 

In [54]:
for artist, lp in lyrics_pages.items() :
    assert(len(set(lp)) > 20) 

In [55]:
# Let's see how long it's going to take to pull these lyrics 
# if we're waiting `5 + 10*random.random()` seconds 
for artist, links in lyrics_pages.items() : 
    print(f"For {artist} we have {len(links)}.")
    print(f"The full pull will take for this artist will take {round(len(links)*10/3600,2)} hours.")

For jellyroll we have 175.
The full pull will take for this artist will take 0.49 hours.
For linkinpark we have 233.
The full pull will take for this artist will take 0.65 hours.


## Part 2: Pulling Lyrics

Now that we have the links to our lyrics pages, let's go scrape them! Here are the steps for this part. 

1. Create an empty folder in our repo called "lyrics". 
1. Iterate over the artists in `lyrics_pages`. 
1. Create a subfolder in lyrics with the artist's name. For instance, if the artist was Cher you'd have `lyrics/cher/` in your repo.
1. Iterate over the pages. 
1. Request the page and extract the lyrics from the returned HTML file using BeautifulSoup.
1. Use the function below, `generate_filename_from_url`, to create a filename based on the lyrics page, then write the lyrics to a text file with that name. 


In [57]:
def generate_filename_from_link(link) :
    
    if not link :
        return None
    
    # drop the http or https and the html
    name = link.replace("https://", "").replace("http://", "").replace(".html", "")

    name = name.replace("/lyrics/","")
    
    # Replace useless chareacters with UNDERSCORE
    name = name.replace("://","").replace(".","_").replace("/","_")
    
    # tack on .txt
    name = name + ".txt"
    return(name)


In [58]:
# Make the lyrics folder here. If you'd like to practice your programming, add functionality 
# that checks to see if the folder exists. If it does, then use shutil.rmtree to remove it and create a new one.

if os.path.isdir("lyrics"):
    # If it exists, remove it
    shutil.rmtree("lyrics")
    print("Existing 'lyrics' folder removed.")

# Create a new 'lyrics' folder
os.mkdir("lyrics")
print("New 'lyrics' folder created.")

Existing 'lyrics' folder removed.
New 'lyrics' folder created.


In [59]:
url_stub = "https://www.azlyrics.com" 
start = time.time()

total_pages = 0 

for artist, links in lyrics_pages.items():
    artist_folder = os.path.join("lyrics", artist)
    
    # Create a subfolder for each artist
    if not os.path.exists(artist_folder):
        os.makedirs(artist_folder)

    for link in links:
        try:
            # Fetch the lyrics page
            r = requests.get(link, headers={'User-Agent': 'Mozilla/5.0'})
            time.sleep(5 + 10 * random.random())  # Sleep to avoid server overload
            
            # Parse the lyrics page
            soup = BeautifulSoup(r.content, 'html.parser')
            
            # Try to find the song title
            title_div = soup.find('div', class_='ringtone')
            if title_div:
                title = title_div.find_next('b').text.strip()
            else:
                title = "Unknown Title"
                print(f"Warning: Could not find the title on {link}")

            # Extract the lyrics (lyrics are usually in a div without class or id)
            lyrics_div = soup.find('div', class_=None, id=None)
            lyrics = lyrics_div.get_text(separator="\n").strip() if lyrics_div else "Lyrics not found."

            # Generate filename from link
            filename = generate_filename_from_link(link)
            file_path = os.path.join(artist_folder, filename)

            # Write title and lyrics to the file
            with open(file_path, 'w', encoding='utf-8') as f:
                f.write(f"{title}\n\n{lyrics}")
                print(f"Saved lyrics for '{title}' to {file_path}")

            # Increment the total pages counter
            total_pages += 1
        
        except Exception as e:
            print(f"Error fetching {link}: {e}")

Saved lyrics for '"On My Way"' to lyrics\jellyroll\www_azlyrics_comjellyroll_onmyway.txt
Saved lyrics for '"Close To The Edge"' to lyrics\jellyroll\www_azlyrics_comjellyroll_closetotheedge.txt
Saved lyrics for '"Falling"' to lyrics\jellyroll\www_azlyrics_comjellyroll_falling.txt
Saved lyrics for '"Papers And Lines"' to lyrics\jellyroll\www_azlyrics_comjellyroll_papersandlines.txt
Saved lyrics for '"Dream While I'm Awake"' to lyrics\jellyroll\www_azlyrics_comjellyroll_dreamwhileimawake.txt
Saved lyrics for '"Farewell"' to lyrics\jellyroll\www_azlyrics_comjellyroll_farewell.txt
Saved lyrics for '"After The Rain"' to lyrics\jellyroll\www_azlyrics_comjellyroll_aftertherain.txt
Saved lyrics for '"Dream While I'm Awake"' to lyrics\jellyroll\www_azlyrics_comjellyroll_dreamwhileimawake.txt
Saved lyrics for '"Bad Times Roll"' to lyrics\jellyroll\www_azlyrics_comjellyroll_badtimesroll.txt
Saved lyrics for '"Goodnight Shirley"' to lyrics\jellyroll\www_azlyrics_comjellyroll_goodnightshirley.txt
Sa

In [60]:
print(f"Total run time was {round((time.time() - start)/3600,2)} hours.")

Total run time was 1.49 hours.


---

# Evaluation

This assignment asks you to pull data by scraping www.AZLyrics.com.  After you have finished the above sections , run all the cells in this notebook. Print this to PDF and submit it, per the instructions.

In [61]:
# Simple word extractor from Peter Norvig: https://norvig.com/spell-correct.html
def words(text): 
    return re.findall(r'\w+', text.lower())

## Checking Lyrics 

The output from your lyrics scrape should be stored in files located in this path from the directory:
`/lyrics/[Artist Name]/[filename from URL]`. This code summarizes the information at a high level to help the instructor evaluate your work. 

In [62]:
artist_folders = os.listdir("lyrics/")
artist_folders = [f for f in artist_folders if os.path.isdir("lyrics/" + f)]

for artist in artist_folders : 
    artist_files = os.listdir("lyrics/" + artist)
    artist_files = [f for f in artist_files if 'txt' in f or 'csv' in f or 'tsv' in f]

    print(f"For {artist} we have {len(artist_files)} files.")

    artist_words = []

    for f_name in artist_files : 
        with open("lyrics/" + artist + "/" + f_name) as infile : 
            artist_words.extend(words(infile.read()))

            
    print(f"For {artist} we have roughly {len(artist_words)} words, {len(set(artist_words))} are unique.")


For jellyroll we have 170 files.
For jellyroll we have roughly 85738 words, 5178 are unique.
For linkinpark we have 230 files.
For linkinpark we have roughly 69112 words, 4003 are unique.
